In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [4]:
llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
# llm=ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
prompt=ChatPromptTemplate.from_template(
                        template="Tell me a joke about {topic}")

In [6]:
chain=prompt|llm|StrOutputParser()
chain.invoke(input={'topic':"Bears"})

'Why did the bear cross the playground?\n\nTo get to the other slide!'

In [7]:
chainRitzStyle=RunnablePassthrough.assign(topic=itemgetter("question")|prompt).assign(result=itemgetter('topic')|llm).assign(output=itemgetter('result')|StrOutputParser())|RunnableLambda(lambda x:x['output'])

In [9]:
chainRitzStyle.invoke(input={'question':"Bears"})

"Why did the bear get a bad grade in school?\n\nBecause he couldn't bear to read!"

<h3> Coercion</h3>

In [12]:
analysisPrompt=ChatPromptTemplate.from_template(
    template="Is this a funny joke? {joke}"
)

In [13]:
combinedChain={"joke":chainRitzStyle}|analysisPrompt|llm|StrOutputParser()

In [14]:
combinedChain.invoke(input={'question':"Bears"})

'Yes, that\'s a funny joke! Here\'s why:\n\n*   **Pun:** The humor comes from the pun on the word "parking" and "paw-king". It\'s unexpected and plays on the bear\'s paws.\n*   **Absurdity:** The image of a bear meticulously "paw-king" silverware is silly and incongruous. It creates a funny mental picture.\n*   **Relatability:** Everyone knows what "parking" is, and the unexpected twist with the bear makes it humorous.\n*   **Simple and Clean:** The joke is short, easy to understand, and doesn\'t require any special knowledge.\n\nOverall, it\'s a lighthearted, pun-based joke that relies on a funny image and wordplay.'

<h3> Using the Pipe Method</h3>

In [15]:
composedChainWithPipe=RunnablePassthrough.assign(joke=chainRitzStyle).pipe(analysisPrompt).pipe(llm).pipe(StrOutputParser())

In [16]:
composedChainWithPipe.invoke(input={'question':"Bears"})

'Yes, that\'s a funny joke! Here\'s why:\n\n*   **Play on Words:** The humor comes from the double meaning of "polar." It refers both to polar bears (who live in cold, icy environments) and to a chemical property (polar molecules dissolve in water).\n*   **Unexpected Twist:** You expect a joke about a bear to be something related to its size, strength, or habitat, not its chemical composition. The unexpected connection is what makes it amusing.\n*   **Clean Humor:** It\'s a pun, making it universally appealing and safe for all audiences.\n\nSo, it\'s a classic example of a pun-based joke that relies on wordplay for its humor.'